In [60]:
# external packages
from pathlib import Path
import matplotlib.pyplot as plt 
from importlib import reload
import matplotlib
%matplotlib qt
import numpy as np
matplotlib.style.use('default')

In [61]:
# local modules and packages
import get_indicators
reload(get_indicators)
from get_indicators import add_indicators_to_raw

In [62]:
# Read in data with indicators
data_with_indicators = add_indicators_to_raw(filepath='..\Data\RawData\EURUSDi1440.csv',save_to_disk=True)
data_with_indicators.head()

c:\forexmachine\env\lib\site-packages\ta\trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
c:\forexmachine\env\lib\site-packages\ta\trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


,Date,Time,Open,High,Low,Close,Volume,chikou_span,volume_adi,volume_obv,...,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,others_dr,others_dlr,others_cr
0,2011.10.14,00:00,1.37765,1.38935,1.37231,1.38772,79276,1.35390,64109.347418,79276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.826084,NaN,0.000000
1,2011.10.16,00:00,1.38780,1.38856,1.38602,1.38775,1313,1.34631,64584.922221,80589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002162,0.002162,0.002162
2,2011.10.17,00:00,1.38655,1.39139,1.37254,1.37371,82980,1.34577,-8094.112262,-2391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.011710,-1.016862,-1.009570
3,2011.10.18,00:00,1.37371,1.38169,1.36526,1.37513,92915,1.35238,10624.612632,90524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.103370,0.103316,-0.907244
4,2011.10.19,00:00,1.37511,1.38688,1.37249,1.37602,89933,1.35250,-35185.595151,180457,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.064721,0.064700,-0.843110


In [63]:
def get_index_range(df,date1,date2):
    i1 = None
    i2 = None
    for i in range(len(df)):
        if df['Date'][i] == date1:
            i1 = i
            break     
    for i in range(i1, len(df)):
        if df['Date'][i] == date2:
            i2 = i
            break
    return i1, i2

In [64]:
# date format 'yyyy.mm.dd'
def show_data_from_range(df, date1, date2, main_indicators = [], sub_indicators = []):
    start, stop = get_index_range(df, date1, date2)
    data_range = df.iloc[start:stop+1]
    chart_count = len(sub_indicators) + 1
    fig, ax = plt.subplots(chart_count,1,sharex='col')
    
    for indicator in main_indicators:
        pass
    
    for indicator in sub_indicators:
        pass
    
    ax.plot(data_range.Close.to_list(), label='Close',color='brown')
    ax.plot(data_range.trend_visual_ichimoku_a.to_list(), label='Senkou-Span a',linestyle='--',color='green')
    ax.plot(data_range.trend_visual_ichimoku_b.to_list(), label='Senkou-Span b',linestyle='--',color='red')
    ax.fill_between(np.arange(len(data_range)),data_range.trend_visual_ichimoku_a,
                    data_range.trend_visual_ichimoku_b,alpha=0.2,color='green',
                    where=(data_range.trend_visual_ichimoku_a > data_range.trend_visual_ichimoku_b))
    ax.fill_between(np.arange(len(data_range)),data_range.trend_visual_ichimoku_a,
                    data_range.trend_visual_ichimoku_b,alpha=0.2,color='red',
                    where=(data_range.trend_visual_ichimoku_a <= data_range.trend_visual_ichimoku_b))
    ax.plot(data_range.trend_ichimoku_conv.to_list(), label='Tenkan-Sen (conversion)',color='cyan')
    ax.plot(data_range.trend_ichimoku_base.to_list(), label='Kijun Sen (base)',color='blue')
    ax.plot(data_range.chikou_span.to_list(), label='chikou span',linestyle=':',color='orange')

    ax.set_xticks(np.arange(len(data_range)))
    #     condition = data_range.index % 10 == 0
    #     condition[-1] = True
    #     x_labels = np.where(condition, data_range['Date'], None)
    x_labels = data_range['Date']
    ax.axvline(stop-start-5,color='red')
    ax.set_xticklabels(x_labels,rotation=90)
    ax. set_title('Ichimoku Kinko Hyo')
    ax.legend()
    plt.show()

def add_ichimoku(ax):

In [65]:
show_data_from_range(data_with_indicators, '2020.04.29', '2020.06.05')